In [1]:
#Importing the libraries
import numpy as np
import pandas as pd

In [2]:
#loading Dataset and picking only useful columns
df_lan = pd.read_excel("../Dataset/Input_Dataset/DDW-C18-0000.xlsx", header = [1])
df_lan = df_lan[((df_lan.iloc[:, 4] == "Total") & (df_lan.iloc[:, 3] == "Total"))].iloc[:, [0, 2, 5, 8]]
df_lan = df_lan.rename(columns={"Area Name" : "Name"})
df_lan.head()

,State,Name,Number speaking second language,Number speaking third language
4,00,INDIA,314988770,86009580
34,01,JAMMU & KASHMIR,6183190,2096220
64,02,HIMACHAL PRADESH,1242705,347286
94,03,PUNJAB,13035224,7829913
124,04,CHANDIGARH,579920,321979


In [3]:
#loading Dataset and picking only useful columns
df_cen = pd.read_excel("../Dataset/Input_Dataset/DDW_PCA0000_2011_Indiastatedist.xlsx")
df_cen = df_cen[((df_cen["TRU"] == "Total") & ((df_cen["Level"] == "India") | (df_cen["Level"] == "STATE")))].iloc[:, [7, 10]]
df_cen["Name"] = df_cen["Name"].apply(lambda x : x.upper())
df_cen.head()

,Name,TOT_P
0,INDIA,1210854977
3,JAMMU & KASHMIR,12541302
72,HIMACHAL PRADESH,6864602
111,PUNJAB,27743338
174,CHANDIGARH,1055450


In [4]:
#Merging df_lan and df_cen 
df_final = pd.merge(df_lan, df_cen, on = "Name")

#Calculating the number number of people speaking exactly one language
df_final["Number speaking first language"] = df_final["TOT_P"] - df_final["Number speaking second language"]
df_final["Number speaking second language"] = df_final["Number speaking second language"] - df_final["Number speaking third language"]

#Calculating the percentage 
df_final["percentage_exact_one_language"] = df_final["Number speaking first language"] * 100 / df_final["TOT_P"]
df_final["percentage_exact_two_language"] = df_final["Number speaking second language"] * 100 / df_final["TOT_P"]
df_final["percentage_exact_three_language"] = df_final["Number speaking third language"] * 100 / df_final["TOT_P"]

#Droping the extra columns
df_final.drop(["Number speaking first language", "Number speaking second language", "Number speaking third language", "TOT_P"], axis = 1, inplace=True)
df_final.drop("Name", axis = 1, inplace = True)

#Renaming the column name
df_final.columns = ["state-code", "percent-one", "percent-two", "percent-three"]
df_final

,state-code,percent-one,percent-two,percent-three
0,00,73.986251,18.910538,7.103211
1,01,50.697384,32.588084,16.714533
2,02,81.896911,13.044005,5.059084
3,03,53.01494,18.762382,28.222678
4,04,45.054716,24.43896,30.506324
5,05,82.572823,15.565026,1.862151
6,06,77.802617,17.580012,4.617371
7,07,59.392197,32.517275,8.090528
8,08,89.097948,9.433381,1.468671
9,09,88.552454,10.153499,1.294046


In [5]:
df= df_final.copy()

In [6]:
#Calculating the ratio
df["Ratio2to1"] = df["percent-two"] / df["percent-one"]
df.sort_values(by = ["Ratio2to1"], inplace=True)
df.head()

,state-code,percent-one,percent-two,percent-three,Ratio2to1
8,08,89.097948,9.433381,1.468671,0.105877
9,09,88.552454,10.153499,1.294046,0.114661
22,22,86.752019,11.59378,1.654201,0.133643
23,23,86.494406,11.773516,1.732078,0.136119
2,02,81.896911,13.044005,5.059084,0.159273


In [7]:
#Storing the last and top three
l = []
for i in list(df["state-code"])[-1:-4:-1]:
    l.append(i)
for i in  list(df["state-code"])[0:3]:
    l.append(i)

In [8]:
df_final = pd.DataFrame(l, columns=["state/ut"])
df_final

,state/ut
0,35
1,30
2,12
3,08
4,09
5,22


In [9]:
#Storing the final result
df_final.to_csv("../Dataset/Output_Dataset/2-to-1-ratio.csv", index=False)